In [1]:
base = '/u/ebanner/Classes/nlp/Project/irony-classifier'

In [ ]:
base = '/u/npockrus/NLP/finalProject/venv/src/irony-classifier'

In [2]:
cd /{base}/data/conservative/features/text+sentiment+subreddit+label/

/v/filer4b/v20q001/ebanner/Classes/nlp/Project/irony-classifier/data/conservative/features/text+sentiment+subreddit+label


# Load Comments 

In [3]:
import pickle
import scipy
import numpy as np

with open('features.p', 'r') as f:
    data = pickle.load(f)

subreddits = np.array([data[sentence]['subreddits'] for sentence in sorted(data)])
sentiments = np.array([data[sentence]['sentiment'] for sentence in sorted(data)])
punctuations = np.array([data[sentence]['punctuation'] for sentence in sorted(data)])
xs = np.array([ sentence.encode('utf-8') for sentence in sorted(data) ])
ys = np.array([ data[sentence]['label'] for sentence in sorted(data) ])

In [4]:
# concatenate features
# features = [ x + ' ' + punct + ' ' + (' SUBREDDITx'.join(subreddit_list)).encode('utf-8') for subreddit_list, x, punct in zip(subreddits, xs, punctuations)]

# Uncomment this and comment the one above to reproduce byron's results
features = [ x + ' ' + punct for x, punct in zip(xs, punctuations)]

print len(xs)
    

2748


In [26]:
features

['\n\n\n(Though in the old days, a communist was whatever the party said it was ) ',
 '\n\n\nHere is the handout he refers to.... ',
 '\n\n\nSo many different people to hate for this. ',
 "\n\n\nSo when CEO's do it, it's wrong and evil, but when the fed does it to keep Obama and the democrats from looking bad spending all this money it's swept under the bus lol\n ",
 '\n\n\nThe problem is, people who advocate for civil rights are not advocating for all of them, just for themselves, or whoever they are guilted into advocating for. ',
 "\n\n\noh and please don't correct my post of the improper use of capitalization because [that be racist! PUNCxEXCLAMATION_POINT",
 "\n\n\noh sorry, he wouldn't so obviously that fuckhead in the crackhouse won't care. ",
 '\n\n"Shovel-ready was not as shovel-ready as we expected... ',
 '\n\n&gt; Both of those programs, however, are public, with no profit motive. ',
 "\n\n&gt; Insurance policies that are offered must now meet minimum standards, covering mor

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=50000, binary=True, ngram_range=(1,2), stop_words="english")
#print subreddits
X = vectorizer.fit_transform(features)

In [7]:
print len(xs)
print len(ys)
vectorizer.get_feature_names()

2748
2748


[u'000',
 u'000 000',
 u'000 continues',
 u'000 defense',
 u'000 federal',
 u'000 home',
 u'000 just',
 u'000 people',
 u'000 puncxuppercase',
 u'000 year',
 u'02',
 u'02 eye',
 u'03',
 u'04',
 u'04 25',
 u'040',
 u'040 yr',
 u'05',
 u'05 puncxuppercase',
 u'050',
 u'050 yr',
 u'07',
 u'07 20',
 u'08',
 u'08 franciscos',
 u'09',
 u'09 10',
 u'09 id',
 u'10',
 u'10 000',
 u'10 09',
 u'10 28',
 u'10 29',
 u'10 federal',
 u'10 iran',
 u'10 is_obama_creating_a_martial',
 u'10 making',
 u'10 minutes',
 u'10 percent',
 u'10 richest',
 u'10 total',
 u'10 years',
 u'100',
 u'100 53',
 u'100 federal',
 u'100 honest',
 u'100 pay',
 u'100 puncxquestion_mark',
 u'100 yearly',
 u'100k',
 u'100k know',
 u'11',
 u'11 03',
 u'11 12',
 u'11 13',
 u'11 26',
 u'11 420',
 u'11 inside',
 u'118',
 u'118 raise',
 u'11e2',
 u'11e2 9008',
 u'11was',
 u'11was inside',
 u'12',
 u'12 03',
 u'12 05',
 u'120',
 u'120 month',
 u'1200',
 u'1200 month',
 u'1297',
 u'1297 preexisting',
 u'13',
 u'13 05',
 u'13 2013',
 

# 5-Fold Test

In [10]:
from sklearn.cross_validation import KFold

kf = KFold(len(xs), n_folds=10, shuffle=True)

# Predictions

In [11]:
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import SGDClassifier

recalls, precisions, f_measures = [], [], []
for train, test in kf:
    # Train svm
    svm = SGDClassifier(loss="hinge", penalty="l2", class_weight="auto", alpha=.01)
    parameters = { 'alpha': [.001, .01,  .1] }
    clf = GridSearchCV(svm, parameters, scoring='f1')
    clf.fit(X[train], ys[train])

    # Make predictions
    predictions = clf.predict(X[test])
    
    for test, vector, prediction, actual in zip(xs[test], X[test], predictions, ys[test]):
        print '========================================================================'
        print
        print 'Comment: {}'.format(test)
        print 'Vectorized as: {}'.format(vector)
        print 'Prediction: {}'.format(prediction)
        print 'Was really: {}'.format(actual)
        print


Comment: 


So many different people to hate for this.
Vectorized as:   (0, 12107)	1
  (0, 4558)	1
  (0, 7305)	1
  (0, 12062)	1
  (0, 4548)	1
Prediction: -1
Was really: -1


Comment: 

3) The society that by and large doesn't object to this objectification of women,?
Vectorized as:   (0, 18348)	1
  (0, 11374)	1
  (0, 11372)	1
  (0, 4812)	1
  (0, 9039)	1
  (0, 15527)	1
  (0, 18342)	1
  (0, 11373)	1
  (0, 11371)	1
  (0, 4789)	1
  (0, 9037)	1
  (0, 15523)	1
  (0, 13335)	1
Prediction: -1
Was really: -1


Comment: 

9.
Vectorized as: 
Prediction: -1
Was really: -1


Comment: 

And please oh please stop with the "enough is enough" and "talk about reckless" and so on.
Vectorized as:   (0, 16421)	1
  (0, 15985)	1
  (0, 11468)	1
  (0, 13760)	1
  (0, 16419)	1
  (0, 15969)	1
  (0, 11459)	1
Prediction: -1
Was really: -1


Comment: 

Anyway, I thought Obama's schtick was that he is a pragmatic, not an absolutist leader.
Vectorized as:   (0, 461)	1
  (0, 12732)	1
  (0, 14819)	1
  (0, 11284)	1
  (0,

# Most Indicative Negative Features

In [12]:
sgd_clf = clf.best_estimator_
feat_names = vectorizer.get_feature_names()
coefs_with_fns = sorted(zip(sgd_clf.coef_[0], feat_names))
coefs_with_fns[:20]

[(-0.028838975643920614, u'think'),
 (-0.025304739417086001, u'going'),
 (-0.020326757892413342, u'gt'),
 (-0.020326757892413342, u'http'),
 (-0.017422935336354313, u'insurance'),
 (-0.016178439955186159, u'year'),
 (-0.014519112780295286, u'years'),
 (-0.014385190705156434, u'puncxuppercase'),
 (-0.01327461739912713, u'com'),
 (-0.01327461739912713, u'day'),
 (-0.01327461739912713, u'new'),
 (-0.012859785605404414, u'www'),
 (-0.012859785605404411, u'law'),
 (-0.012444953811681696, u'care'),
 (-0.012444953811681696, u'http www'),
 (-0.01161529022423626, u'point'),
 (-0.011200458430513534, u'believe'),
 (-0.011200458430513534, u'public'),
 (-0.01058637672012085, u'liberal'),
 (-0.0099559630493453628, u'better')]

# Most Indicative Features

In [14]:
coefs_with_fns[-20:]

[(0.014917612613635233, u'obamacare puncxuppercase'),
 (0.015332444407357952, u'omg'),
 (0.015332444407357952, u'staggering'),
 (0.015332444407357952, u'staggering 93'),
 (0.015332444407357952, u'stories'),
 (0.017971689205215961, u'obama'),
 (0.020294093971647869, u'need'),
 (0.020509675848700608, u'oh'),
 (0.020924507642423328, u'guys'),
 (0.021339339436146045, u'knows'),
 (0.021538589352816041, u'sure'),
 (0.021953421146538761, u'racist'),
 (0.024226830031822341, u'good'),
 (0.025272075496320542, u'idea'),
 (0.031693802318831316, u'news'),
 (0.034597624874890369, u'yeah'),
 (0.037899947264289337, u'obamacare'),
 (0.042646014951526284, u'just'),
 (0.05169065461151192, u'puncxquestion_mark'),
 (0.076629558116023774, u'puncxexclamation_point')]

# Performance

## Byron's Results: Precision=0.315 (0.261, 0.380), Recall=0.496 (0.446, 0.548), F-Measure=0.383 (0.330, 0.412)

In [25]:
import sklearn

precisions, recalls, f_measures = [], [], []
for train, test in kf:
    # Train svm
    svm = SGDClassifier(loss="hinge", penalty="l2", class_weight="auto")
    parameters = { 'alpha': [.001, .01,  .1] }
    clf = GridSearchCV(svm, parameters, scoring='f1')
    clf.fit(X[train], ys[train])

    # Make predictions
    predictions = clf.predict(X[test])
    
    # Record statistics
    precision, recall, f_measure, _ = sklearn.metrics.precision_recall_fscore_support(ys[test], predictions, average='binary')
    precisions.append(precision)
    recalls.append(recall)
    f_measures.append(f_measure)
    
print 'Precisions: {}'.format(precisions)
print 'Recalls: {}'.format(recalls)
print 'F-Measures: {}'.format(f_measures)
print
print 'Mean Precision: {}'.format(np.mean(precisions))
print 'Mean Recall: {}'.format(np.mean(recalls))
print 'Mean F-Measure: {}'.format(np.mean(f_measures))

Precisions: [0.086206896551724144, 0.12244897959183673, 0.042944785276073622, 0.14285714285714285, 0.10714285714285714, 0.10526315789473684, 0.1111111111111111, 0.10062893081761007, 0.077777777777777779, 0.14492753623188406]
Recalls: [0.41666666666666669, 0.27272727272727271, 0.69999999999999996, 0.25, 0.29999999999999999, 0.72727272727272729, 0.125, 0.80000000000000004, 0.5, 0.58823529411764708]
F-Measures: [0.14285714285714288, 0.16901408450704225, 0.080924855491329481, 0.18181818181818182, 0.15789473684210525, 0.18390804597701149, 0.11764705882352941, 0.1787709497206704, 0.13461538461538464, 0.23255813953488372]

Mean Precision: 0.104130917525
Mean Recall: 0.467990196078
Mean F-Measure: 0.158000858019
